In [28]:
import os
import numpy as np
import pandas as pd

import joblib

from utils import get_features, text_embedding

from keras.models import load_model
from sentence_transformers import SentenceTransformer

In [31]:
current_path=os.getcwd().replace("\\", "/").replace("c:", "C:")
data_path=current_path+"/wavFiles/"
model_path=current_path+"/model/"

In [32]:
label=pd.read_csv(current_path+"/wav_df.csv")
label.rename(columns={"filename":"wav_id", "label":"final_label"}, inplace=True)

In [41]:
model = load_model(model_path+'my_model.h5')
encoder = joblib.load(model_path+'encoder.pkl')
scaler = joblib.load(model_path+'scaler.pkl')
with open(model_path+'text_model_name.txt', 'r') as file:
    text_model_name = file.read().strip()

In [50]:
file_name='301_AUDIO_112268.0_113068.0_SPLIT.wav'
new_audio_path = data_path+file_name
new_audio_features = get_features(new_audio_path)
new_audio_features_df = pd.DataFrame([new_audio_features])

new_audio_features_df['wav_id']=file_name
new_audio_features_df['final_label']=label[label['wav_id']==file_name]['final_label'].values[0]
new_audio_features_df['sentence']=label[label['wav_id']==file_name]['sentence'].values[0]

In [51]:
txt_embed = text_embedding(model_name = text_model_name)
new_audio_features_df = txt_embed.transform(new_audio_features_df)

# 특징 표준화
new_audio_features_scaled = scaler.transform(new_audio_features_df)

# 모델에 맞게 차원 확장
new_audio_features_scaled = np.expand_dims(new_audio_features_scaled, axis=2)

# 예측 수행
prediction = model.predict(new_audio_features_scaled)
predicted_label = np.argmax(prediction, axis=1)

1/1 [==============================] - 0s 21ms/step


In [52]:
predicted_label

array([0], dtype=int64)